In [1]:
# Step 1: Generator network

import keras
from keras import models, layers
import numpy as np

latent_dim = 32
height, width, n_channels = (32, 32, 3)

generator_input = keras.Input(shape=(latent_dim,))
x = layers.Dense(units=128*16*16)(generator_input)
# Using LeakyReLu, activate function, to prevent sparse gradient problem.
x = layers.LeakyReLU()(x)
# Convert shape of feature map as (16, 16, 128)
x = layers.Reshape((16, 16, 128))(x)
x = layers.Conv2D(filters=256, kernel_size=5, padding='same')(x)
x = layers.LeakyReLU()(x)
# Upsampling
x = layers.Conv2DTranspose(filters=256, kernel_size=4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(filters=256, kernel_size=5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(filters=256, kernel_size=5, padding='same')(x)
x = layers.LeakyReLU()(x)
# Final output has shape as (32, 32, 3). It is same as image array's shape.
# If GAN is trained well, the generator output generated image like real image.
x = layers.Conv2D(filters=n_channels, kernel_size=7, activation='tanh', padding='same')(x)

generator = keras.models.Model(input=generator_input, output=x)
generator.summary()

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32768)             0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 32, 32, 256)       1048832   
__________

/home/guhwanbae/anaconda3/envs/gu-keras/lib/python3.6/site-packages/ipykernel_launcher.py:29: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


In [2]:
# Step 2: Discriminator network

# Input of the discriminator has a shape as (height, width, n_channels), image.
discriminator_input = keras.Input(shape=(height, width, n_channels))
x = layers.Conv2D(filters=128, kernel_size=3)(discriminator_input)
x = layers.LeakyReLU()(x)
# Downsampling
x = layers.Conv2D(filters=128, kernel_size=4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(filters=128, kernel_size=4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(filters=128, kernel_size=4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)
# Dropout layer to prevent local minimize.
x = layers.Dropout(rate=0.45)(x)
# Binary classification
x = layers.Dense(units=1, activation='sigmoid')(x)

discriminator = keras.models.Model(input=discriminator_input, output=x)
discriminator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 6, 6, 128)         0         
__________

/home/guhwanbae/anaconda3/envs/gu-keras/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [3]:
discriminator_optimizer = keras.optimizers.RMSprop(lr=0.0007,
                                                   clipvalue=1.0,
                                                   decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer,
                      loss='binary_crossentropy')
discriminator.trainable = False

In [4]:
# Step 3: GAN, generative adversarial network

gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))

gan = keras.models.Model(input=gan_input, output=gan_output)

gan_optimizer = keras.optimizers.RMSprop(lr=0.00045,
                                         clipvalue=1.0,
                                         decay=1e-8)

gan.compile(optimizer=gan_optimizer,
            loss='binary_crossentropy')

gan.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 32)                0         
_________________________________________________________________
model_1 (Model)              (None, 32, 32, 3)         6264579   
_________________________________________________________________
model_2 (Model)              (None, 1)                 790913    
Total params: 7,055,492
Trainable params: 6,264,579
Non-trainable params: 790,913
_________________________________________________________________


/home/guhwanbae/anaconda3/envs/gu-keras/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("mo...)`
  


In [5]:
# Step 4: Train a GAN

from keras.datasets import cifar10
from keras import preprocessing
import os


'''
CIFAR10 Classes:

0 : airplane
1 : automobile
2 : bird
3 : cat
4 : deer
5 : dog
6 : frog
7 : horse
8 : ship
9 : truck
'''
class_index = {0:'airplane',
               1:'automobile',
               2:'bird',
               3:'cat',
               4:'deer',
               5:'dog',
               6:'frog',
               7:'horse',
               8:'ship',
               9:'truck'}

(train_data, train_targets), (test_data, test_targets) = cifar10.load_data()

# Choose a label randomly.
label = np.random.randint(low=0, high=10)

# Choose a single class.
train_data = train_data[train_targets.flatten() == label]

class_name = class_index[label]
n_samples = train_data.shape[0]
print('Choose a class %s, label %d' % (class_name, label))
print('The number of samples =', n_samples)

train_data = train_data.reshape((n_samples, height, width, n_channels))
train_data = train_data.astype('float32') / 255.0

Choose a class truck, label 9
The number of samples = 5000


In [6]:
import time

save_dir = './gan-generated-' + class_name
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

iterations = 10000
batch_size = 20

begin_epoch = time.time()
start = 0
for step in range(iterations):
    # Choose a sample in latent space.
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

    # Decoding fake images.
    fake_images = generator.predict(random_latent_vectors)
    
    stop = start + batch_size
    real_images = train_data[start:stop]
    
    combined_images = np.concatenate([fake_images, real_images])
    
    labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
    labels += 0.05 * np.random.random(labels.shape)
    
    discriminator_loss = discriminator.train_on_batch(combined_images, labels)
    
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
    
    misleading_targets = np.zeros((batch_size, 1))
    gan_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
    
    start += batch_size
    
    if start > (n_samples - batch_size):
        start = 0
    if (step % 100) == 0:
        end_epoch = time.time()
        
        gan.save_weights('gan.h5')
        print('-'*80)
        print('epoch = %d, %s sec' % (step, (end_epoch - begin_epoch)))
        print('Discriminator loss =', discriminator_loss)
        print('GAN loss =', gan_loss)
        
        # Save a fake iamge.
        img = preprocessing.image.array_to_img(fake_images[0] * 255.0, scale=False)
        img.save(os.path.join(save_dir, 'fake_' + class_name + '_' + str(step) + '.png'))
        # Save a real image.
        img = preprocessing.image.array_to_img(real_images[0] * 255.0, scale=False)
        img.save(os.path.join(save_dir, 'real_' + class_name + '_' + str(step) + '.png'))
        
        begin_epoch = time.time()

/home/guhwanbae/anaconda3/envs/gu-keras/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


--------------------------------------------------------------------------------
epoch = 0, 4.232402324676514 sec
Discriminator loss = 0.6995174
GAN loss = 0.67411184
--------------------------------------------------------------------------------
epoch = 100, 26.36231541633606 sec
Discriminator loss = 0.6672346
GAN loss = 1.2210934
--------------------------------------------------------------------------------
epoch = 200, 26.460723400115967 sec
Discriminator loss = 0.7090889
GAN loss = 0.77718747
--------------------------------------------------------------------------------
epoch = 300, 26.51597237586975 sec
Discriminator loss = 0.6875516
GAN loss = 0.8394782
--------------------------------------------------------------------------------
epoch = 400, 26.482739210128784 sec
Discriminator loss = 0.6877143
GAN loss = 0.749619
--------------------------------------------------------------------------------
epoch = 500, 26.79546570777893 sec
Discriminator loss = 0.7561489
GAN loss = 0

--------------------------------------------------------------------------------
epoch = 4900, 26.48762607574463 sec
Discriminator loss = 0.6682862
GAN loss = 0.8036044
--------------------------------------------------------------------------------
epoch = 5000, 26.474180698394775 sec
Discriminator loss = 0.62473184
GAN loss = 0.89237815
--------------------------------------------------------------------------------
epoch = 5100, 26.48740315437317 sec
Discriminator loss = 0.70363945
GAN loss = 0.8042613
--------------------------------------------------------------------------------
epoch = 5200, 26.8161301612854 sec
Discriminator loss = 0.71689135
GAN loss = 0.8072203
--------------------------------------------------------------------------------
epoch = 5300, 26.686927556991577 sec
Discriminator loss = 0.70812607
GAN loss = 0.81318474
--------------------------------------------------------------------------------
epoch = 5400, 26.699541568756104 sec
Discriminator loss = 0.6910339

--------------------------------------------------------------------------------
epoch = 9800, 26.611857414245605 sec
Discriminator loss = 0.5236918
GAN loss = 2.353588
--------------------------------------------------------------------------------
epoch = 9900, 26.597471952438354 sec
Discriminator loss = 0.64740384
GAN loss = 1.0290115
